In [0]:
import os
import xml.etree.ElementTree as ET
import copy

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/SVT'
# path where images directory is stored 

In [0]:
def parse_annotation(data):
  '''
  parse train and test xml files to generate annotations
  inputs -
    data - string denoting filename
  outputs -
    all_images - list of dicts containing annotations for each image
  '''
  # path = '/content/drive/My Drive/SVT'  # path where images directory is stored
  fpath = os.path.join(path, data)  # global path being used
  tree = ET.parse(fpath)  
  root = tree.getroot() # tagset
  all_images = []

  for child in root.findall('image'): # image
    info = {}
    for child_1 in child.findall('imageName'):
      info['filename'] = os.path.join(path, child_1.text)
    for child_2 in child.findall('lex'):
      info['lex'] = child_2.text
    for child_3 in child.findall('Resolution'):
      temp = child_3.attrib
      info['height'] = int(temp['y'])
      info['width'] = int(temp['x'])

    info['object'] = []
    for child_4 in child.findall('taggedRectangles'):
      
      for gchild_1 in child_4.findall('taggedRectangle'):
        bbox = {}
        temp2 = gchild_1.attrib
        xmin = int(temp2['x'])
        ymin = int(temp2['y'])
        w = int(temp2['width'])
        h = int(temp2['height'])

        # stored in xmin, ymin, xmax, ymax format
        bbox['xmin'] = xmin
        bbox['ymin'] = ymin
        bbox['xmax'] = xmin + w
        bbox['ymax'] = ymin + h
        for gg_child_1 in gchild_1.findall('tag'):
          bbox['label'] = gg_child_1.text
          
        info['object'].append(bbox)
    all_images.append(info)
  return all_images

In [0]:
def slice_dataset(list_gainer, list_loser, train = 0.6, val = 0.2, test = 0.2):
  '''
  slice dataset to create required train:val:testsplit
  inputs -
    list_gainer - train list created above (100 images) 
    list_loser - test list created above (249 images), from which we transfer images to train list
    train, val, test - ratios of their distribution in total
  outputs - 
    list_gainer, list_created, new_list_loser - lists created in the given ratio
  '''
  assert(train + val + test == 1)
  total = len(list_gainer) + len(list_loser)
  train_contrib = int(total * train - len(list_gainer)) # portion from list_loser to contrib for train; 0 tp train_contrib
  # assert(train_contrib > 0)
  val_contrib = int(total * val + train_contrib) # portion from list_loser to contrib for val; train_contrib to val_contrib

  new_loser = copy.deepcopy(list_loser)
  list_created = []
  for _ in range(train_contrib):
    ele = new_loser.pop(0)
    list_gainer.append(ele)

  for _ in range(train_contrib, val_contrib):
    ele1 = new_loser.pop(0)
    list_created.append(ele1)  # was ele earlier, lead to faulty valid_dataset
  
  print(total, '=', len(list_gainer), '+', len(list_created), '+', len(new_loser))
  return list_gainer, list_created, new_loser


In [6]:
train_images = parse_annotation('train.xml')
test_images = parse_annotation('test.xml')
print(len(train_images), len(test_images))
print('After slicing:')
train_images, val_images, test_images = slice_dataset(train_images, test_images, 0.7, 0.15, 0.15)

100 249
After slicing:
349 = 244 + 52 + 53


In [7]:
import pprint 
pp = pprint.PrettyPrinter(indent=3)
pp.pprint(train_images[0])

{  'filename': '/content/drive/My Drive/SVT/img/14_03.jpg',
   'height': 880,
   'lex': 'LIVING,ROOM,THEATERS,KENNY,ZUKE,DELICATESSEN,CLYDE,COMMON,ACE,HOTEL,PORTLAND,ROSE,CITY,BOOKS,STUMPTOWN,COFFEE,ROASTERS,RED,CAP,GARAGE,FISH,GROTTO,SEAFOOD,RESTAURANT,AURA,RESTAURANT,LOUNGE,ROCCO,PIZZA,PASTA,BUFFALO,EXCHANGE,MARK,SPENCER,LIGHT,FEZ,BALLROOM,READING,FRENZY,ROXY,SCANDALS,MARTINOTTI,CAFE,DELI,CROWSENBERG,HALF',
   'object': [  {  'label': 'LIVING',
                   'xmax': 611,
                   'xmin': 375,
                   'ymax': 328,
                   'ymin': 253},
                {  'label': 'ROOM',
                   'xmax': 814,
                   'xmin': 639,
                   'ymax': 348,
                   'ymin': 272},
                {  'label': 'THEATERS',
                   'xmax': 1120,
                   'xmin': 839,
                   'ymax': 370,
                   'ymin': 283}],
   'width': 1280}


In [0]:
# save lists for future use 
import pickle

with open(os.path.join(path, 'train_images_.pickle'), 'wb') as handle:
  pickle.dump(train_images, handle)

with open(os.path.join(path, 'val_images_.pickle'), 'wb') as handle:
  pickle.dump(val_images, handle)

with open(os.path.join(path, 'test_images_.pickle'), 'wb') as handle:
  pickle.dump(test_images, handle)
